# Using Detectron2 for detecting budding spots in yeast
Follow these steps to use a pre-trained model of Detectron2 on yeast data to detect budding spots in your microscopy images.
## Step 1: Setup your Google Drive
- Create a folder called Detectron2_budding_yeast in the root of your Google Drive
- Upload this ipynb file in the folder
- Upload the model model_final.pth in a folder model
- Upload your tif files in a folder new_data_tif

## Step 2: Open this file from Google Drive using Google Colaboratory and mount Drive by running the cell below
- Install Google Colaboratory using Google Workspace Marketplace and link to Google Drive if neccesary
- Accept the access request from Google Colaboratory for Google Drive prompted when running the code cell below



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Detectron2_budding_yeast

## Step 3: Install Detectron2 and setup by running the three code cells below
- Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
- See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions

In [ ]:
import sys, os, distutils.core

!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# import tif library
import tifffile as tiff


# Step 4: Inference with config parameters

In [ ]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # learning rate
cfg.SOLVER.MAX_ITER = 500    # run for 500 iterations
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # confidence required to label a budding spot
predictor = DefaultPredictor(cfg)

# Step 5: Visualize performance on new data

In [ ]:
output_folder = 'output_masks/'
os.makedirs(output_folder, exist_ok=True)

tif_file_path = 'new_data_tif/'
tif_files = [os.path.join(tif_file_path, img_name) for img_name in os.listdir(tif_file_path)]

# Create a dictionary to store the masks for each cell
cell_masks = {}

for cell, tif_file in enumerate(tif_files):
    # Load the 4D TIFF image
    img = tiff.imread(tif_file)

    # Initialize an empty list to store masks for each cell
    cell_masks[cell] = []

    # Get the shape of the first mask to ensure all masks have the same shape
    mask_shape = None

    # Iterate through the time (t) and z-stacks
    for t in range(img.shape[0]):
        cell_masks[cell].append([])
        for z in range(img.shape[1]):
            # Extract the frame for the current time (t) and z-stack
            page = img[t, z, :, :]

            # Normalize pixel values to the range [0, 255]
            image = cv2.normalize(page, None, 0, 255, cv2.NORM_MINMAX)

            # Convert the frame to BGR
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Annotate the data
            outputs = predictor(image)
            masks = outputs["instances"].pred_masks.cpu().numpy()

            # Initialize an empty mask array with the same shape as the image
            union_mask = np.zeros_like(image[:,:,0])

            # Iterate through each mask and update the union_mask
            for mask in masks:
                union_mask = np.logical_or(union_mask, mask)

            # Append the union mask to the list of masks for the cell
            cell_masks[cell][t].append(cv2.normalize(union_mask.astype(np.uint8), None, 0, 255, cv2.NORM_MINMAX))

            # Update mask shape if not initialized yet
            if mask_shape is None:
                mask_shape = union_mask.shape

    # Check if mask shape is initialized
    if mask_shape is not None:
        # Convert masks to numpy array and check for any discrepancies in shape
        stacked_masks = np.array(cell_masks[cell])
        if not all(mask.shape == mask_shape for mask in stacked_masks.flatten()):
            print(f"Warning: Inconsistent mask shapes in cell {cell+1}.")
        # Save the 4D TIFF file for each cell with z layers per t timestep
        tiff.imwrite(os.path.join(output_folder, f"cell{cell+1}_masks.tif"), stacked_masks)
    else:
        print(f"Warning: No masks found for cell {cell+1}.")

